In [1]:
from datasets import load_dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from transformers import AutoTokenizer
from data.dataset import VQADataset
from data.collector import Collector
from model.Qwen3VL import Qwen3VL
from model.Config import Qwen3VLConfig

tokenizer = AutoTokenizer.from_pretrained("/media/qba/Data/Project/DeepLearning/Model/Qwen3-0.6B")

data = load_dataset("parquet", data_files="/media/qba/Data/Project/DeepLearning/Dataset/the_cauldron/tqa/train-00000-of-00001-c15be8aed9c93862.parquet")["train"]
image_processor = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

dataset = VQADataset(data, tokenizer, image_processor, 49)
collector = Collector(tokenizer.pad_token_id,"cpu")
data_loader = DataLoader(dataset, batch_size=2, collate_fn=collector)

data1 = next(iter(data_loader))

In [2]:
data1["images"][0].shape

torch.Size([1, 3, 224, 224])

In [3]:
config = Qwen3VLConfig.from_pretrained("/media/qba/Data/Project/DeepLearning/Build VLM from scratch/src/Qwen3VL")
model = Qwen3VL(config)#.to("cuda")
out = model(**data1)
print(out)


(tensor([[ 4.9193e-01, -4.3981e-01, -6.1097e-02,  ..., -1.1683e+00,
         -5.9019e-01, -2.6650e-01],
        [ 4.9594e-01,  8.8556e-01, -1.3475e-01,  ..., -1.0242e+00,
         -3.8373e-01,  6.3107e-03],
        [ 8.4393e-01,  6.3141e-01, -5.2264e-01,  ...,  3.1402e-01,
         -7.6301e-01, -2.1951e-01],
        ...,
        [ 2.5659e-01,  1.1087e+00,  4.7638e-01,  ...,  2.7668e-01,
          4.9186e-01, -4.2844e-01],
        [-5.3715e-01,  1.2138e+00, -2.4492e-02,  ..., -7.4251e-01,
          9.9593e-02,  2.0317e-01],
        [-3.4082e-01,  9.4105e-01, -4.0758e-02,  ...,  7.7099e-04,
         -6.3091e-01, -3.7275e-02]], grad_fn=<ViewBackward0>), tensor(12.3010, grad_fn=<NllLossBackward0>))
